# Тестовое задание Data Analyst | Задание 2

* Тестовое задание состоит из нескольких задач, которые необходимо реализовать на языке Python.
* Допускается применение дополнительных пакетов, будет плюсом распараллелить задачи для ускорения выполнения там где это требуется.
* Все тестовые датасеты загружаем и процессим (делаем выборки, джойны, фильтруем и т.д.) в коде.
* Если у вас не получилось сделать какие-то шаги, но вы понимаете все остальное - пропускайте их и делайте то  в чем разбираетесь.
* Результат выполнения - ссылка на github с jupiter notebook. Высылаем все ответы / ссылки на fediaeva@skytecgames.com. Так же напишите сколько часов у вас ушло на выполнение.
* При возникновении вопросов по тестовому заданию - пишем туда же, на  fediaeva@skytecgames.com
* Необходимо сделать минимум 3 задания на ваш выбор. Чем больше заданий сделали, тем лучше.
* Тестовый датасет прилагается к письму sqlite файлом `testcase.db.zip`

---

In [1]:
%%capture no-display
!pip install scipy statsmodels

In [2]:
import numpy as np
import pandas as pd

import scipy.stats as st
import statsmodels.stats.power as smp

---

## 2 Сравнение групп платящих игроков

В мобильной игре около 10% игроков совершает платежи в первый месяц с момента установки игры (база расчета - 500 игроков). Геймдизайнеры разработали обновление которое по их экспертной оценке должно увеличить процент плательщиков с 10% до 11%. Цель эксперимента - подтвердить или опровергнуть их гипотезу.

1.	Опишите оптимальный по вашему мнению дизайн эксперимента
2.	Рассчитайте длительность эксперимента при условии что каждый день в игру приходит около 100 новых игроков.
3.	Самостоятельно сгенерируйте датасет с около 10% плательщиков (контроль) и рассчитайте 95% HDI / CI.
4.	Сгенерируйте несколько вариантов экспериментальной группы (хуже, лучше, без эффекта), рассчитайте силу эффекта, HDI / CI и ваши выводы для каждого варианта.
5.	Решите задачу 2 разными подходами: frequentist / bayesian

Необходимо все ответы сопроводить комментариями почему вы выбрали ту или иную методику расчета, алгоритм и т.д.

Исходные данные:

В первый месяц с момента установки игры | Процент | Игроков
-|-|-
Всего игроков | 100% | 5000
Платежи совершают | 10% | 500
Увеличение процента плательщиков после обновления | 11% | 550

<br>

Гипотезы:
* Нулевая ($ H_0 $) - Обновление игры не увеличит процент плательщиков, и он останется на уровне 10%
* Альтернативная ($ H_1 $) - Обновление игры увеличит процент плательщиков с 10% до 11%

<br>

### 2.1 Описание оптимального дизайна эксперимента

><br>
><b>Обоснование</b>
>
>В качестве оптимального дизайна эксперимента подходит A/B-тестирование, поскольку оно позволяет понять, приведёт ли нововведение к улучшению результатов.
><br><br>

Дизайн эксперимента:
1. Разделить новых игроков с момента установки игры на 2 группы случайным образом с вычисленным минимальным числом пользователей в каждой (по 4913 игроков):
    * Контрольная группа - без обновления (доля плательщиков = 10% = 0.1)
    * Экспериментальная группа - с обновлением (ожидаемая доля плательщиков = 11% = 0.11)
2. Отслеживать изменение доли плательщиков в каждой группе в течение вычисленной длительности эксперимента (98 дней)
3. Сравнить долю плательщиков в обеих группах с учётом уровня статистической значимости ($ \alpha = 0.05 $):
    * Разница статистически значима - обновление увеличило процент плательщиков
    * Разница статистически не значима - обновление не увеличило процент плательщиков

<br>

### 2.2 Расчёт длительности эксперимента при условии, что каждый день в игру приходит около 100 новых игроков

Описание эксперимента:

Свойство | Условное<br>обозначение | Значение
-|-|-
Конверсия до обновления | $ CR_{base} $ | 0.1
Конверсия после обновления | $ CR_{new} $ | 0.11
Уровень статистической значимости | $ \alpha $ | 0.05
Бета | $ \beta  $ | 0.2
Статистическая мощность | $ power $ | $ power = 1 - \beta = 1 - 0.2 = 0.8 $
Стандартное нормальное распределение для $ \alpha $ | $ Z_{\alpha} $ | 0.8289 (по таблице нормального распределения)<br>при двустороннем уровне значимости $ 1 - \frac {\alpha} {2} = 1 - \frac {0.05}{2} = 1 - 0.025 = 0.975 $
Стандартное нормальное распределение для $ power$ | $ Z_{\beta} $ | 0.7881 (по таблице нормального распределения)

Расчёт минимального размера выборки:
$$ n = (Z_{\alpha} + Z_{\beta})^2 \times \frac { CR_{base} \times (1 - CR_{base}) + CR_{new} \times (1 - CR_{new}) } { (CR_{new} - CR_{base})^2 } = \\ \quad \\ (0.8289 + 0.7881)^2 \times \frac { 0.1 \times (1 - 0.1) + 0.11 \times (1 - 0.11) } { (0.11 - 0.1)^2 } = \\ \quad \\ (1.617)^2 \times \frac { 0.1 \times 0.9 + 0.11 \times 0.89 } { (0.01)^2 } = 2.614689 \times \frac { 0.09 + 0.0979 } {0.0001} = \\ \quad \\ 2.614689 \times \frac {0.1879} {0.0001} = 2.614689 \times 1879 = \sim 4913 $$

<br>

Свойство | Условное<br>обозначение | Значение
-|-|-
Новых игроков в день | $ u_{day} $ | ~100
Минимальный размер выборки | $ n_{min} $ | 4913
Количество групп в эксперименте | $ k $ | 2 (контрольная + экспериментальная)
Размер выборки | $ N $ | $ N = n_{min} \times k = 4913 \times 2 = 9826 $
Длительность эксперимента | $ days $ | $ days = \frac {N}{ u_{day} } = \frac {9826}{ \sim 100 } = \sim 98 $ дней

><br>
><b>Вывод</b>
>
>Длительность эксперимента при условии, что каждый день в игру приходит около 100 новых игроков, составит 98 дней.
<br><br>

<br>

### 2.3 Генерация датасета с около 10% плательщиков (контроль) и расчёт 95% HDI / CI

Зададим долевое соотношение состояний игроков (плательщик или нет):

In [3]:
# creating data with ration between payers and non-payers
data_CR = pd.DataFrame.from_dict({
    'is_payer': [0, 1], 
    'ratio': [0.9, 0.1]
})

# checking data
data_CR

,is_payer,ratio
0,0,0.9
1,1,0.1


Генерация датасета с около 10% плательщиков (контроль):

In [4]:
def generate_data(data: pd.DataFrame, col: str) -> pd.Series:
    '''Generate data with specified ratio
        
        Args:
        - data (pd.DataFrame) - initial data
        - col (str) - column name to take ratio values

        Returns:
        - pd.Series - generated sequence of payer and non-payer status of players
    '''

    sequence = np.random.choice(
        data['is_payer'], 
        size=N_players, 
        p=data[col]
    )

    return pd.Series(sequence)


def get_stats(data: pd.DataFrame) -> pd.DataFrame:
    '''Get statistics of data: count and ratio

        Args:
        - data (pd.DataFRame) - initial DataFrame

        Returns:
        - pd.DataFrame - statistics of data
    '''

    vc = pd.DataFrame(data.value_counts())
    vc_norm = pd.DataFrame(data.value_counts(normalize=True))

    stats = vc.join(vc_norm)
    stats.columns = ['count', 'ratio']

    return stats


# total players
N_players = 5000

# creating control data with 10% of payers
data = generate_data(data_CR, 'ratio')

# getting control data statistics
data_stats = get_stats(data)

# checking results
data_stats

,count,ratio
0,4506,0.9012
1,494,0.0988


Расчёт 95% HDI / CI:

In [5]:
# calculating binomial confidence interval
lower, upper = st.binom.interval(0.95, N_players, data_stats.loc[1, 'ratio'])

# checking results
print(f'95% HDI/CI: {lower / N_players}/{upper / N_players}')

95% HDI/CI: 0.0906/0.1072


><br>
><b>Вывод</b>
>
>Существует вероятность 95%, что доверительный интервал содержит игроков, совершивших платежи в первый месяц после установки игры.
>
>Таким образом обновление игры может увеличить процент плательщиков с 10% до 11%.
><br><br>

<br>

### 2.4 Генерация нескольких вариантов экспериментальной группы (хуже, лучше, без эффекта), расчёт силы эффекта, HDI / CI и выводы для каждого варианта

Добавление долевых соотношений состояний игроков:

In [6]:
# # creating data with ration between payers and non-payers
data_CR['better'] = [1 - 0.12, 0.12]

# creating control data with 9% of payers
data_CR['worser'] = [1 - 0.09, 0.09]

# creating control data with no effect
data_CR['no_effect'] = [1 - 0.1, 0.1]

# checking results
data_CR

,is_payer,ratio,better,worser,no_effect
0,0,0.9,0.88,0.91,0.9
1,1,0.1,0.12,0.09,0.1


Генерация датасета с около 12% (лучше), 9% (хуже) и 10% (без эффекта) плательщиков:

In [7]:
# creating experimental group with 12% of payers
data_better = generate_data(
    data=data_CR[['is_payer', 'better']], 
    col='better'
)

# creating experimental group with 9% of payers
data_worser = generate_data(
    data=data_CR[['is_payer', 'worser']], 
    col='worser'
)

# creating experimental group with no effect
data_no_effect = generate_data(
    data=data_CR[['is_payer', 'no_effect']], 
    col='no_effect'
)

# getting experimental group statistics
data_stats = data_stats.join(get_stats(data_better), rsuffix='_better')
data_stats = data_stats.join(get_stats(data_worser), rsuffix='_worser')
data_stats = data_stats.join(get_stats(data_no_effect), rsuffix='_no_effect')

# checking results
data_stats

,count,ratio,count_better,ratio_better,count_worser,ratio_worser,count_no_effect,ratio_no_effect
0,4506,0.9012,4397,0.8794,4521,0.9042,4513,0.9026
1,494,0.0988,603,0.1206,479,0.0958,487,0.0974


Расчёт силы эффекта и HDI/CI:

In [8]:
# calculating binomial confidence interval
for col in ['ratio_better', 'ratio_worser', 'ratio_no_effect']:
    ratio_control = data_stats.loc[1, 'ratio']
    ratio_experiment = data_stats.loc[1, col]
    ratio_effect = abs(ratio_experiment - ratio_control)

    object = smp.TTestPower()

    power = object.solve_power(
        effect_size=ratio_effect,
        nobs=1000,
        alpha=0.05,
        power=None,
        alternative='two-sided'
    )
    
    lower, upper = st.binom.interval(0.95, N_players, ratio_experiment)

    print(f'Power of {col}:', round(power, 4))
    print(f'95% HDI/CI of {col}: {lower / N_players}/{upper / N_players}')
    print()

Power of ratio_better: 0.1059
95% HDI/CI of ratio_better: 0.1116/0.1296

Power of ratio_worser: 0.051
95% HDI/CI of ratio_worser: 0.0878/0.104

Power of ratio_no_effect: 0.0502
95% HDI/CI of ratio_no_effect: 0.0892/0.1056



><br>
><b>Вывод</b>
>
>Чем выше мощность, тем больше вероятность теста выявить реальные значения:
>* Лучше (12%) - Если истинный эффект существует, то тест обнаружил его в 12%
>
>Существует 95% вероятность, что обновление игры увеличит процент плательщиков до 11-13%. 
>* Хуже (9%) - Если истинный эффект существует, то тест обнаружил его в 6%
>
>Существует 95% вероятность, что обновление игры уменьшит процент плательщиков до 9-10%. 
>* Без эффекта (10%) - Если истинный эффект существует, то тест обнаружил его в 5%
>
>Существует 95% вероятность, что обновление игры уменьшит процент плательщиков до 9-10%. 
><br><br>

<br>

### 2.5 Решение задачи 2.2 разными подходами: frequentist / bayesian

**frequentist**

\-

**bayesian**

\-